In [2]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import computational_graph
from chainer import serializers

In [3]:
# 分類器インスタンスの生成（これがないとモデルを読み込めない）
# Classifier クラスは定義したニューラルネットを分類器として簡単に利用するためのクラス
# MLPクラスのインスタンスを引数として渡す
from functions import MLP
model = L.Classifier(MLP())

ModuleNotFoundError: No module named 'functions'

In [16]:
#modelのよみこみ
#(file_name, 読み込みたいモデルに与える変数名)

serializers.load_npz('mnist.model', model)
print('OK')

OK


In [17]:
#ここから実際に分析に入る
from PIL import Image
img = Image.open('num_9.jpg')
img.show()

In [25]:
from functions import convert_img
data_img = convert_img(img)
draw_digit(data_img)

NameError: name 'np' is not defined

In [35]:
result = predict(model, data_img)
print(result)
if(result == 9):
    print('GOOD')
else:
    print('NOOO')

3
NOOO


In [33]:
#MLPクラス（学習モデルを定義する）

class MLP(chainer.Chain):
    #ニューラルネットの構造を定義
    def __init__(self):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(784, 900)
            self.l2 = L.Linear(900, 1000)
            self.l3 = L.Linear(1000, 500)
            self.l4 = L.Linear(500, 10)
            
    #ニューラルネットの計算規則を定義
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        h3 = F.relu(self.l3(h2))
        return self.l4(h3)

In [23]:
#モデルを利用して予測をする関数を定義
def predict(model, x):
    #データ数が１の場合は、バッチサイズ分の次元を追加
    if len(x.shape) == 1:
        pred = model.predictor(x[None, ...]).data.argmax()
    #データ数が2以上の場合はそのまま
    else:
        pred = model.predicor(x).data.argmax(axis=1)
    return pred

In [24]:
#手書き文字変換の関数
#sample_img = 読み込む写真
def convert_img(sample_img):
    sample_img_re = sample_img.convert('L').resize((28, 28)) #白黒反転と28*28にリサイズ
    array_sample_img = np.asarray(sample_img_re, dtype=np.float32)/255 #chainer用にarray化と正規化
    array_sample_img = 1 -array_sample_img #写真が白の場合の対応
    train_img = []
    for x in range(28):
        for y in range(28):
            train_img.append(array_sample_img[x, y])
    train_img = np.array(train_img)
    return train_img